<a href="https://colab.research.google.com/github/hchristiaan/TrainCNN4SignatureVerification/blob/master/ProjectCode_Test_SigComp2009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

import keras
import keras.backend as K
import tensorflow as tf
from keras import applications
from keras.models import Model
from keras.layers import Flatten, Dense, Input,concatenate
from keras.optimizers import Adam
from keras.models import load_model, model_from_json

import random

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!unzip '/content/drive/Team Drives/inputdataset/input.zip' -d /content

Archive:  /content/drive/Team Drives/inputdataset/input.zip
   creating: /content/handwritten-signatures/
   creating: /content/handwritten-signatures/sample_Signature/
   creating: /content/handwritten-signatures/sample_Signature/forged/
  inflating: /content/handwritten-signatures/sample_Signature/forged/NFI-00101014.png  
  inflating: /content/handwritten-signatures/sample_Signature/forged/NFI-00101019.png  
  inflating: /content/handwritten-signatures/sample_Signature/forged/NFI-00101025.PNG  
  inflating: /content/handwritten-signatures/sample_Signature/forged/NFI-00101026.PNG  
  inflating: /content/handwritten-signatures/sample_Signature/forged/NFI-00101027.PNG  
  inflating: /content/handwritten-signatures/sample_Signature/forged/NFI-00101028.PNG  
  inflating: /content/handwritten-signatures/sample_Signature/forged/NFI-00101029.PNG  
  inflating: /content/handwritten-signatures/sample_Signature/forged/NFI-00101030.PNG  
  inflating: /content/handwritten-signatures/sample_Signa

In [0]:
gentr="/content/sigcomp-2009-train/genuine"
forgtr="/content/sigcomp-2009-train/forgeries"

gent="/content/sigcomp-2009/genuines"
forgt="/content/sigcomp-2009/forgeries"

In [0]:
img_width, img_height, channels = 224, 224, 3

dim = (img_width, img_height)

def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

def returnimages(path,img):
    image=cv2.imread(path+"/"+ img)                  #bringing the image
    image=cv2.resize(image, (img_width, img_height))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=to_rgb(image).reshape(1,img_width, img_height,3)/255.0       #resizing and normalizing    
    return image     

def getfiles(num,gen,forg):
    genList=os.listdir(gen)
    forgList=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    
    forgArr,genArr=[],[]
    for i in forgList:
        if i.endswith(c+".png"):
            forgArr=forgArr+[i]
        elif i.endswith(c+".PNG"):
            forgArr=forgArr+[i]
    for i in genList:
        if i.endswith(c+".png"):
            genArr=genArr+[i]
        elif i.endswith(c+".PNG"):
            genArr=genArr+[i]
    return genArr.pop(),forgArr,genArr

def getfiles2(num):
    genList=os.listdir(gentr)
    forgList=os.listdir(forgtr)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    forgArr,genArr=[],[]
    for i in forgList:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            forgArr=forgArr+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            forgArr=forgArr+[i]
    for i in genList:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            genArr=genArr+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            genArr=genArr+[i]
    return genArr.pop(),forgArr,genArr

In [0]:
def triplet_loss(y_true, y_pred):
    alpha = 0.5
    anchor, positive, negative =y_pred[0,0:512], y_pred[0,512:1024], y_pred[0,1024:1536]
    
    positive_distance = K.mean(K.square(anchor - positive),axis=-1)
    negative_distance = K.mean(K.square(anchor - negative),axis=-1)
    return K.mean(K.maximum(0.0, positive_distance - negative_distance + alpha))

In [0]:
model = applications.vgg16.VGG16(weights='imagenet', include_top=False, pooling='max')

In [0]:
for layer in model.layers[:5]:
    layer.trainable = False

In [0]:
anchor_in = Input(shape=(img_width, img_height, channels))
pos_in = Input(shape=(img_width, img_height, channels))
neg_in = Input(shape=(img_width, img_height, channels))

# Defining output of model
anchor_out = model(anchor_in)
pos_out = model(pos_in)
neg_out = model(neg_in)
merged_vector = concatenate([anchor_out, pos_out, neg_out],axis=1)

In [0]:
model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)

In [0]:
model.compile(optimizer=Adam(lr=0.000005),loss=triplet_loss)

In [0]:
def generator():
    #Need to alter these numbers being tested
    x=["0.001", "0.002","0.003","0.004", "0.005", "0.006", "0.007",
       "0.008", "0.009", "0.010", "0.011", "0.012"]
  #  x=["0.001", "0.004", "0.006", "0.010"]

    for k in x:
        anc,neg,pos=getfiles2(k)
        frac=0.95    
        inds = set(random.sample(list(range(len(neg))), int(frac*len(neg))))
        neg = [n for i,n in enumerate(neg) if i not in inds]
    
        for i in range(len(neg)):
            for j in range(len(pos)):
                anchor=returnimages(gentr,anc)
                positive=returnimages(gentr,pos[j])
                negative=returnimages(forgtr,neg[i])
               # yield ([anc,pos[j],neg[i]])
                yield ([anchor,positive,negative],[0])

In [31]:
for x in range(1):
    model.fit_generator(generator(),steps_per_epoch=32,epochs=10)

Epoch 1/10
32/32 [==============================] - 4s 139ms/step - loss: 0.2456
Epoch 2/10
32/32 [==============================] - 3s 85ms/step - loss: 0.1044
Epoch 3/10
32/32 [==============================] - 3s 85ms/step - loss: 0.0302
Epoch 4/10
32/32 [==============================] - 3s 85ms/step - loss: 0.0188
Epoch 5/10
32/32 [==============================] - 3s 85ms/step - loss: 0.0000e+00
Epoch 6/10
32/32 [==============================] - 3s 85ms/step - loss: 0.0073
Epoch 7/10
32/32 [==============================] - 3s 85ms/step - loss: 0.1201
Epoch 8/10
32/32 [==============================] - 3s 84ms/step - loss: 0.0033
Epoch 9/10
32/32 [==============================] - 3s 84ms/step - loss: 0.0000e+00
Epoch 10/10
32/32 [==============================] - 3s 84ms/step - loss: 0.0409


In [32]:
tneg,tpos=0,0
#Need to alter these numbers being tested
x=[0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]
for k in x: #the id of signatures you want to check
    #print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    tneg=tneg+len(neg)
    tpos=tpos+len(pos)
print(tneg,tpos)

564 181


In [0]:
model.save_weights('model_weights.h5')

In [34]:
forg_passed=0
gen_flagged=0
#Need to alter these numbers being tested
x=[0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    
    anchor=returnimages(gent,anc)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor,positive,anchor])
        a, p, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        dist=np.linalg.norm(a-p)
        #print("positive distance is ",dist)
        if(dist>32):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor,negative,anchor])
        a, n, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist=np.linalg.norm(a-n)
        #print("negative distance is ",dist)
        if(dist>32):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)

When k is  0.016
gen flagged -  35.53665 file name is -  NFI-01609016.png
gen flagged -  34.591858 file name is -  NFI-01612016.png
forg passed -  31.79929 file name is -  NFI-08506016.png
When k is  0.018
gen flagged -  34.916687 file name is -  NFI-01810018.png
gen flagged -  50.21405 file name is -  NFI-01807018.png
gen flagged -  33.35818 file name is -  NFI-01801018.png
gen flagged -  57.01723 file name is -  NFI-01811018.png
gen flagged -  55.459637 file name is -  NFI-01802018.png
When k is  0.024
gen flagged -  33.155468 file name is -  NFI-02411024.png
When k is  0.033
forg passed -  29.579985 file name is -  NFI-02705033.png
forg passed -  31.3193 file name is -  NFI-01704033.png
forg passed -  28.265896 file name is -  NFI-05906033.png
forg passed -  27.74149 file name is -  NFI-01706033.png
forg passed -  27.933718 file name is -  NFI-01703033.png
forg passed -  30.35545 file name is -  NFI-01603033.png
forg passed -  29.17296 file name is -  NFI-02702033.png
forg passed - 

In [0]:
def getfilest(num,gen,forg):
    genList=os.listdir(gen)
    forgList=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    
    forgArr,genArr=[],[]
    for i in forgList:
        if i.endswith(c+".png"):
            forgArr=forgArr+[i]
        elif i.endswith(c+".PNG"):
            forgArr=forgArr+[i]
    for i in genList:
        if i.endswith(c+".png"):
            genArr=genArr+[i]
        elif i.endswith(c+".PNG"):
            genArr=genArr+[i]
    return genArr.pop(),genArr.pop(),forgArr,genArr

In [36]:
forg_passed=0
gen_flagged=0
x=[0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc1,anc2,neg,pos=getfilest(k,gent,forgt)
    
    anchor1=returnimages(gent,anc1)
    anchor2=returnimages(gent,anc2)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor1,positive,anchor2])
        a1, p, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        dist1=np.linalg.norm(a1-p)
        dist2=np.linalg.norm(a2-p)
        dist=(dist1+dist2)/2
        #print("positive distance is ",dist)
        if(dist>32):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor1,negative,anchor2])
        a1, n, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist1=np.linalg.norm(a1-n)
        dist2=np.linalg.norm(a2-n)
        #print("negative distance is ",dist)
        dist=(dist1+dist2)/2
        if(dist>32):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)

When k is  0.016
gen flagged -  36.7496452331543 file name is -  NFI-01609016.png
gen flagged -  37.105648040771484 file name is -  NFI-01612016.png
When k is  0.018
gen flagged -  45.06507873535156 file name is -  NFI-01810018.png
gen flagged -  42.73736572265625 file name is -  NFI-01807018.png
gen flagged -  43.11723327636719 file name is -  NFI-01801018.png
gen flagged -  43.624996185302734 file name is -  NFI-01811018.png
When k is  0.024
When k is  0.033
forg passed -  27.42725372314453 file name is -  NFI-02705033.png
forg passed -  28.49289894104004 file name is -  NFI-01704033.png
forg passed -  29.60437774658203 file name is -  NFI-05906033.png
forg passed -  28.934242248535156 file name is -  NFI-01706033.png
forg passed -  29.91094970703125 file name is -  NFI-02804033.png
forg passed -  28.303112030029297 file name is -  NFI-01703033.png
forg passed -  30.045238494873047 file name is -  NFI-01603033.png
forg passed -  26.23369598388672 file name is -  NFI-02702033.png
forg